&emsp;&emsp;设置工作目录：

In [1]:
setwd('C:/Users/sirzh/Desktop/R实现/00quanzhouqi/00quanzhouqi')

getwd()

[1] "C:/Users/sirzh/Desktop/R实现/00quanzhouqi/00quanzhouqi"

&emsp;&emsp;利用名义GDP和地区生产总值指数计算实际GDP：

In [2]:
real_gdp <- function (nominal_gdp, gdp_deflator) {
    real_gdp =  rep(1, length = length(nominal_gdp))
    real_gdp[1] = nominal_gdp[1] 
    if (length(nominal_gdp) >= 2) {
        for (i in 2:length(nominal_gdp)) {
            real_gdp[i] = nominal_gdp[i-1] * gdp_deflator[i] / 100   
        }
    }
    return(real_gdp) 
}

&emsp;&emsp;利用 GDP 预测电力消费弹性

In [3]:
data = read.csv(file = 'gdp_electricity.csv', encoding = 'gbk', header = TRUE)

nominal_gdp_sample = data[, 2]
gdp_deflator_sample = data[, 3]
ele_sample = data[, 4]
gdp_deflator_forecast = c(107.5, 107.5, 107.5)

In [4]:
real_gdp_sample = real_gdp(nominal_gdp_sample, gdp_deflator_sample)  # 将名义 GDP 转化为实际 GDP

In [5]:
model <- lm(ele_sample ~ 1 + log(real_gdp_sample))    # 拟合模型
last_gdp = real_gdp_sample[length(real_gdp_sample)]   # 最后一个观测值

&emsp;&emsp;根据给出的 gdp_deflator_forecast 数量设置设置预测数量：

In [6]:
ele_forecast = rep(1,length(gdp_deflator_forecast))   # 生成“虚假”的电力预测值

In [7]:
for (i in 1:length(ele_forecast)) {
    forecast_data = data.frame(real_gdp_sample = c(last_gdp))
    result = predict.lm(model, newdata = forecast_data, interval = "prediction")
    ele_forecast[i] = result[1]
    last_gdp = last_gdp * gdp_deflator_forecast[i] / 100 # 将本期的预测值作为下期预测值的根据
}

results = list(ele_forecast = ele_forecast, real_gdp_sample = real_gdp_sample, first_fit = model)

In [8]:
results$ele_forecast

[1] 3126.050 3208.368 3290.686